In [1]:
import json
import pandas as pd

import psycopg2
import os
from dotenv import load_dotenv
from surprise import SVD, Dataset, Reader,KNNBasic,KNNBaseline,KNNWithMeans
from surprise.model_selection import train_test_split,cross_validate,GridSearchCV
load_dotenv()


True

In [18]:
def connect_database():
    conn = psycopg2.connect(
        host=os.getenv('HOST'),
        dbname=os.getenv('DATABASE_NAME'),
        user=os.getenv('USER_DB'),
        password=os.getenv('PASSWORD'),
        port=os.getenv('PORT')
    )
    return conn

In [19]:
conn = connect_database()
cur = conn.cursor()

query1 = """SELECT uc.id, uc.username, unnested.release_id, r.release_year
FROM user_collection AS uc
JOIN LATERAL unnest(uc.collection_ids) AS unnested(release_id) ON true
JOIN release AS r ON unnested.release_id = r.id;
"""
collection_df = pd.read_sql_query(query1, conn)
collection_df['score'] = 2

/var/folders/41/ry041x6n14l4pdd350dygsfh0000gn/T/ipykernel_10153/1891838526.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  collection_df = pd.read_sql_query(query1, conn)


In [20]:
query2= """
SELECT uw.id, uw.username, unnested.release_id, r.release_year
FROM user_wantlist AS uw
JOIN LATERAL unnest(uw.wantlist_ids) AS unnested(release_id) ON true
JOIN release AS r ON unnested.release_id = r.id;"""
wantlist_df = pd.read_sql_query(query2, conn)
wantlist_df['score'] = 1
cur.close()
conn.close()


/var/folders/41/ry041x6n14l4pdd350dygsfh0000gn/T/ipykernel_10153/3493328829.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  wantlist_df = pd.read_sql_query(query2, conn)


In [21]:
merged_df = pd.concat([collection_df, wantlist_df], ignore_index=True)
merged_df = merged_df.rename(columns={'id':'user_id'})


In [31]:
filtered_df =merged_df[(merged_df['release_year'] >  1980) & (merged_df['release_year'] < 2005)]

filtered_df


,user_id,username,release_id,release_year,score
13,7,AlessandroFischer,5842450,2000.0,2
15,8,daveybonez,4128,2001.0,2
16,9,KRAKE1000,86736,1992.0,2
19,162,Arobelidze,50047,1993.0,2
20,162,Arobelidze,107820,1992.0,2
...,...,...,...,...,...
2173734,526,flaxy,404111,1998.0,1
2173735,526,flaxy,209915,1998.0,1
2173736,526,flaxy,1189622,2002.0,1
2173739,526,flaxy,70732,1996.0,1


In [23]:
print(f"We have {len(merged_df['username'].unique())} unique users")
print(f"We have {len(merged_df['release_id'].unique())} unique releases")

We have 505 unique users
We have 904462 unique releases


In [32]:
reader = Reader(rating_scale=(1, 2))
data = Dataset.load_from_df(filtered_df[['user_id','release_id','score']],reader)

In [33]:
trainset,testset = train_test_split(data,test_size=0.2)
algo = SVD()

In [34]:
algo.fit(trainset)
predictions = algo.test(testset)

In [35]:
N = 10


user_releases = set(merged_df[merged_df['user_id'] == 109]['release_id'].tolist())

user_id = 109 
item_ids = merged_df['release_id'].unique().tolist()
user_ratings = {item_id: algo.predict(user_id, item_id).est for item_id in item_ids if item_id not in user_releases}
recommended_items = sorted(user_ratings, key=user_ratings.get, reverse=True)[:N] 


In [13]:
recommended_items

[1029773,
 120012,
 19725394,
 11159284,
 16081198,
 1165115,
 368781,
 6044574,
 2045891,
 139247]

In [36]:
conn = connect_database()
cur = conn.cursor()
q = f"""
SELECT r.id, r.title, STRING_AGG(ra.artist_name, ', ') as artists, rl.label_name, r.country, r.release_year
FROM release AS r
JOIN release_artist AS ra ON r.id = ra.release_id
JOIN release_label AS rl ON r.id = rl.release_id
WHERE r.id IN {tuple(recommended_items)} AND ra.extra = '0'
GROUP BY r.id, r.title, rl.label_name, r.country, r.release_year
ORDER BY r.title
"""


cur.execute(q)
res = cur.fetchall()

for id,title,artist,label,country,year in res:
    print(f"{title} By  {artist} On  {label}  {country}  {year}")
    

100 Years Later By  Organ Grinda On  Pro-Jex  UK  2001
Casting EP By  Elesbaan On  Audiodrome  Spain  1999
Inner Laugh (Disc.01) By  Roland Klinkenberg On  Lost Language  UK  2000
Keep Your Crossover By  Dual Race On  Syncopate  UK  1997
Rich In Paradise / Going Back To My Roots By  FPI Project On  Rumour Records  UK  1989
Scorchio By  Darren Emerson, Sasha On  Deconstruction  UK  2000
Stealth (12 Inch No.2) By  Way Out West, Kirsty Hawkshaw On  Distinct'ive Breaks  UK  2002
The Sound Of Slacker EP By  Slacker On  Jukebox In The Sky  UK  2001
Throbbing Disco Cat By  Takkyu Ishino On  Ki/oon  Japan  1999
Throbbing Disco Cat By  Takkyu Ishino On  Loopa  Japan  1999
Touch By  Jeroen Schrijvershof On  Search  Netherlands  1997
